In [1]:
require 'countries'
require 'numo/narray'

true

# 80. コーパスの整形

文を単語列に変換する最も単純な方法は，空白文字で単語に区切ることである． ただ，この方法では文末のピリオドや括弧などの記号が単語に含まれてしまう． そこで，コーパスの各行のテキストを空白文字でトークンのリストに分割した後，各トークンに以下の処理を施し，単語から記号を除去せよ．

トークンの先頭と末尾に出現する次の文字を削除: .,!?;:()[]'"
空文字列となったトークンは削除
以上の処理を適用した後，トークンをスペースで連結してファイルに保存せよ．

In [2]:
doc = []
pat = "[#{["\\", "\\.", "\\,", "\\!", "\\?", "\\;", "\\:", "\\(", "\\)", "\\[", "\\]", "\\'", "\""].join()}]"

File.foreach('../data/enwiki-20150112-400-r100-10576.txt').with_index do |line, index|
  p index if index % 50000 == 0
  sentence = line.chomp.split.map {|token|
    token.
      sub(/^#{pat}/, '').
      sub(/#{pat}$/, '').
      gsub(/#{pat}/, '') # ズル
    }.
    reject{|token| token.empty?}
  
  doc << sentence.join("\s") if sentence.size > 5
end

1

0
50000
100000
150000
200000
250000


1

# 81. 複合語からなる国名への対処

英語では，複数の語の連接が意味を成すことがある．例えば，アメリカ合衆国は"United States"，イギリスは"United Kingdom"と表現されるが，"United"や"States"，"Kingdom"という単語だけでは，指し示している概念・実体が曖昧である．そこで，コーパス中に含まれる複合語を認識し，複合語を1語として扱うことで，複合語の意味を推定したい．しかしながら，複合語を正確に認定するのは大変むずかしいので，ここでは複合語からなる国名を認定したい．

インターネット上から国名リストを各自で入手し，80のコーパス中に出現する複合語の国名に関して，スペースをアンダーバーに置換せよ．例えば，"United States"は"United_States"，"Isle of Man"は"Isle_of_Man"になるはずである

In [3]:
original = ISO3166::Country.all.map{|country| country.name.sub(/\s\(.*?\)\s/, '')}.select{|name| name.match(/\s/)}
under_scored = original.map{|name| name.gsub(/\s/, '_')}

doc.each_with_index do |sentence, index|
  p index if index % 50000 == 0
  original.size.times do |i|
    sentence.gsub!(original[i-1], under_scored[i-1])
  end
end

Marshal.dump(doc, open('../data/knock81.dump', 'wb'))

1

0
50000
100000
150000


1

# 82. 文脈の抽出

81で作成したコーパス中に出現するすべての単語ttに関して，単語ttと文脈語ccのペアをタブ区切り形式ですべて書き出せ．ただし，文脈語の定義は次の通りとする．

ある単語ttの前後dd単語を文脈語ccとして抽出する（ただし，文脈語に単語ttそのものは含まない）
単語ttを選ぶ度に，文脈幅ddは{1,2,3,4,5}{1,2,3,4,5}の範囲でランダムに決める．

In [4]:
def window
  (Random.rand * 4.9999).to_i + 1
end

context_list = []

doc.each_with_index do |words, index|
  p index if index % 50000 == 0
  words = words.split(/\s|\n/)
  
  words.size.times.each do |i|
    w = window

    start_from = [0, i - w].max
    end_to = [words.size-1, i + w].min

    contexts = words[start_from...i] + words[(i+1)..end_to]
    contexts.each do |context|
      context_list << [words[i], context]
    end
  end
end

1

0
50000
100000
150000


1

# 83. 単語／文脈の頻度の計測

82の出力を利用し，以下の出現分布，および定数を求めよ．

f(t,c)f(t,c): 単語ttと文脈語ccの共起回数
f(t,∗)f(t,∗): 単語ttの出現回数
f(∗,c)f(∗,c): 文脈語ccの出現回数
NN: 単語と文脈語のペアの総出現回数

In [5]:
# NOTE: Marshal can't dump Hash object with default proc
# count_t = Hash.new{|hash, key| hash[key] = 0}
# count_c = Hash.new{|hash, key| hash[key] = 0}
# count_t_c = Hash.new{|hash, key| hash[key] = Hash.new{|hash2, key2| hash2[key2] = 0}}

count_t = Hash.new
count_c = Hash.new
count_t_c = Hash.new
n = 0

context_list.each_with_index do |(t, c), index|
  p index if index % 10000000 == 0

  # NOTE: alternative to default proc
  count_t[t] = 0 unless count_t[t]
  count_c[c] = 0 unless count_c[c]
  count_t_c[t] = Hash.new unless count_t_c[t]
  count_t_c[t][c] = 0 unless count_t_c[t][c]

  count_t[t] += 1
  count_c[c] += 1
  count_t_c[t][c] += 1
  n += 1
end

1

0
10000000
20000000
30000000
40000000
50000000
60000000


1

# 84. 単語文脈行列の作成

83の出力を利用し，単語文脈行列XXを作成せよ．ただし，行列XXの各要素XtcXtcは次のように定義する．

f(t,c)≥10f(t,c)≥10ならば，Xtc=PPMI(t,c)=max{logN×f(t,c)f(t,∗)×f(∗,c),0}Xtc=PPMI(t,c)=max{log⁡N×f(t,c)f(t,∗)×f(∗,c),0}
f(t,c)<10f(t,c)<10ならば，Xtc=0Xtc=0
ここで，PPMI(t,c)PPMI(t,c)はPositive Pointwise Mutual Information（正の相互情報量）と呼ばれる統計量である．なお，行列XXの行数・列数は数百万オーダとなり，行列のすべての要素を主記憶上に載せることは無理なので注意すること．幸い，行列XXのほとんどの要素は00になるので，非00の要素だけを書き出せばよい．

In [6]:
f = Hash.new
i = 0

token2id = Hash.new
id2token = Hash.new

count_t.keys.each_with_index do |token, id|
  token2id[token] = id
  id2token[id] = token
end

count_t_c.each do |t, hash|
  p i if i % 10000000 == 0
  hash.each do |c, val|
    ppmi1 = Math.log((n*val) / (count_t[t]*count_c[c]))
    f[token2id[t]] = Hash.new unless f[token2id[t]]
    f[token2id[t]][token2id[c]] = [0, ppmi1].max if val >= 10
    i += 1
  end
end

f.reject! {|key, hash| hash.size == 0}
data = [f, token2id]

1

0


1

# 85. 主成分分析による次元圧縮

84で得られた単語文脈行列に対して，主成分分析を適用し，単語の意味ベクトルを300次元に圧縮せよ．

保留

# 86. 単語ベクトルの表示

85で得た単語の意味ベクトルを読み込み，"United States"のベクトルを表示せよ．ただし，"United States"は内部的には"United_States"と表現されていることに注意せよ．

In [7]:
# TODO: I have not done knock85 because of the limitation of Numo::NArray and NMatrix.
#       So, this knock have not been completed yet.

p 'United_States'
p token2id['United_States']
p f[token2id['United_States']]
p 'US'
p token2id['US']
p f[token2id['US']]

1

"United_States"
274
{272=>0, 27=>0.6931471805599453, 44=>0.6931471805599453, 275=>0, 154=>0, 11=>0, 2=>0, 278=>1.0986122886681098, 279=>0, 280=>0, 46=>0, 55=>0, 892=>2.995732273553991, 266=>0.6931471805599453, 217=>0, 1175=>0, 758=>0, 290=>0, 542=>0, 636=>1.9459101490553132, 204=>0, 135=>0, 706=>0, 335=>0.6931471805599453, 1=>0, 5=>0, 128=>0, 1438=>1.0986122886681098, 305=>0, 436=>0, 1742=>1.0986122886681098, 22=>0, 120=>0, 4177=>2.1972245773362196, 4178=>2.9444389791664403, 5840=>2.0794415416798357, 33=>0, 53=>0.6931471805599453, 1610=>2.8903717578961645, 5928=>1.0986122886681098, 529=>0.6931471805599453, 6016=>2.995732273553991, 29=>0, 344=>0, 155=>0, 456=>0.6931471805599453, 4780=>0, 6436=>0.6931471805599453, 218=>0, 494=>0, 30=>0, 903=>0, 5752=>2.9444389791664403, 5753=>2.772588722239781, 1269=>2.995732273553991, 12104=>2.8903717578961645, 652=>0, 1639=>1.0986122886681098, 239=>0, 1240=>0, 696=>2.8903717578961645, 12845=>5.099866427824199, 5964=>4.543294782270004, 3698=>0.693147180

1

# 87. 単語の類似度

85で得た単語の意味ベクトルを読み込み，"United States"と"U.S."のコサイン類似度を計算せよ．ただし，"U.S."は内部的に"U.S"と表現されていることに注意せよ．

In [8]:
def sim(v1, v2)
  v1.transpose.dot(v2) / Math.sqrt(v1.transpose.dot(v1) * v2.transpose.dot(v2))
end

:sim

# 88. 類似度の高い単語10件

85で得た単語の意味ベクトルを読み込み，"England"とコサイン類似度が高い10語と，その類似度を出力せよ．

In [9]:
def distance(mat, v1)
  result = Hash.new
  mat.shape[0].times do |i|
    v2 = mat[i, true]
    similarity = sim(v1, v2)
    result[i] = similarity
  end

  result.sort{|(_, val1), (_, val2)| val2 <=> val1}[1..10]
end

:distance

# 89. 加法構成性によるアナロジー

85で得た単語の意味ベクトルを読み込み，vec("Spain") - vec("Madrid") + vec("Athens")を計算し，そのベクトルと類似度の高い10語とその類似度を出力せよ．

In [10]:
def analogy(mat, v1, v2, v3)
  result = Hash.new
  v4 = v1 - v2 + v3

  mat.shape[0].times do |i|
    v_ = mat[i, true]
    similarity = sim(v4, v_)
    result[i] = similarity
  end

  result.sort{|(_, val1), (_, val2)| val2 <=> val1}[1..10]
end

:analogy